In [2]:
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import pipeline
from accelerate import Accelerator

tokenizer = LlamaTokenizer.from_pretrained("/mnt/disk-1/llama_hf_7B")
model = LlamaForCausalLM.from_pretrained("/mnt/disk-1/llama_hf_7B")

/home/yuxizheng/miniconda3/envs/get_responses/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.12s/it]


In [3]:
batches = ["Answer with only one number from 1 to 7, considering 1 as the least similar and 7 as the most similar: How similar is Alligator and Alligator?",
           "Answer with only one number from 1 to 7, considering 1 as the least similar and 7 as the most similar: How similar is Alligator and Cat?",
           "Answer with only one number from 1 to 7, considering 1 as the least similar and 7 as the most similar: How similar is Dog and Cat?"]

In [6]:

accelerator = Accelerator()
    
model, tokenizer = accelerator.prepare(model, tokenizer)

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=accelerator.device)

responses = pipe(batches, max_new_tokens=20, do_sample=False)
responses = [r[0]['generated_text'] for r in responses]
responses = [','.join(r.split('\n')[1::]) for r in responses]

prompt_and_response = []
for prompt, response in zip(batches, responses):
    prompt_and_response.append([prompt, response])
    
prompt_and_response

[['Answer with only one number from 1 to 7, considering 1 as the least similar and 7 as the most similar: How similar is Alligator and Alligator?',
  'How similar is Alligator and Alligator?,How similar is Alligator and'],
 ['Answer with only one number from 1 to 7, considering 1 as the least similar and 7 as the most similar: How similar is Alligator and Cat?',
  'How similar is Alligator and Cat?,Asked by 1000'],
 ['Answer with only one number from 1 to 7, considering 1 as the least similar and 7 as the most similar: How similar is Dog and Cat?',
  'How similar is Dog and Cat?,The answer is: 0.5555']]